In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
# file_path = '/Users/mymac/Google_Drive/Forex_Robot/'
file_path = '/Users/mymac/'

In [3]:
# ----------------------------------------------------------------------------------------------------
# Get the data
# ----------------------------------------------------------------------------------------------------
# df = pd.read_csv(file_path + 'Oanda_Aud_Usd_M5_2020-2021.csv')
df = pd.read_csv(file_path + 'Oanda_Usd_Cad_M5_2018-2021_test.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

In [4]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2021-12-01 07:00:00   1.27306   1.27361  1.27267    1.27356   1.27328   
1 2021-12-01 07:05:00   1.27358   1.27411  1.27349    1.27409   1.27379   
2 2021-12-01 07:10:00   1.27411   1.27444  1.27392    1.27434   1.27436   
3 2021-12-01 07:15:00   1.27438   1.27469  1.27418    1.27465   1.27456   
4 2021-12-01 07:20:00   1.27462   1.27542  1.27462    1.27542   1.27481   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  \
0   1.27381  1.27290    1.27376   1.27317   1.27371  1.27278    1.27366   
1   1.27437  1.27370    1.27434   1.27368   1.27424  1.27360    1.27422   
2   1.27466  1.27415    1.27455   1.27424   1.27455  1.27404    1.27444   
3   1.27489  1.27437    1.27484   1.27447   1.27479  1.27428    1.27474   
4   1.27562  1.27481    1.27562   1.27472   1.27552  1.27472    1.27552   

   Volume  
0     177  
1     184  
2     179  
3     160  
4     151  
--------------------------

In [5]:
def psar(barsdata, iaf=0.02, maxaf=0.2):
    length = len(barsdata)
    high = list(barsdata['Mid_High'])
    low = list(barsdata['Mid_Low'])
    close = list(barsdata['Mid_Close'])
    psar = close[0:len(close)]
    bull = True
    af = iaf
    hp = high[0]
    lp = low[0]
    for i in range(2, length):
        if bull:
            psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
        else:
            psar[i] = psar[i - 1] + af * (lp - psar[i - 1])
        reverse = False
        if bull:
            if low[i] < psar[i]:
                bull = False
                reverse = True
                psar[i] = hp
                lp = low[i]
                af = iaf
        else:
            if high[i] > psar[i]:
                bull = True
                reverse = True
                psar[i] = lp
                hp = high[i]
                af = iaf
        if not reverse:
            if bull:
                if high[i] > hp:
                    hp = high[i]
                    af = min(af + iaf, maxaf)
                if low[i - 1] < psar[i]:
                    psar[i] = low[i - 1]
                if low[i - 2] < psar[i]:
                    psar[i] = low[i - 2]
            else:
                if low[i] < lp:
                    lp = low[i]
                    af = min(af + iaf, maxaf)
                if high[i - 1] > psar[i]:
                    psar[i] = high[i - 1]
                if high[i - 2] > psar[i]:
                    psar[i] = high[i - 2]
    return psar


def atr(barsdata):
    high_low = barsdata['Mid_High'] - barsdata['Mid_Low']
    high_close = np.abs(barsdata['Mid_High'] - barsdata['Mid_Close'].shift())
    low_close = np.abs(barsdata['Mid_Low'] - barsdata['Mid_Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(14).sum() / 14


def rsi(barsdata, periods=14):
    close_delta = barsdata['Mid_Close'].diff()

    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))

    return rsi

  
def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()

    return adx_smooth


def stoch(high, low, close, lookback=14):
    high_lookback = high.rolling(lookback).max()
    low_lookback = low.rolling(lookback).min()
    slow_k = (close - low_lookback) * 100 / (high_lookback - low_lookback)
    slow_d = slow_k.rolling(3).mean()

    return slow_k, slow_d

def n_macd(macd, macdsignal, lookback=50):
    n_macd = 2 * (((macd - macd.rolling(lookback).min()) / (macd.rolling(lookback).max() - macd.rolling(lookback).min()))) - 1
    n_macdsignal = 2 * (((macdsignal - macdsignal.rolling(lookback).min()) / (macdsignal.rolling(lookback).max() - macdsignal.rolling(lookback).min()))) - 1

    return n_macd, n_macdsignal

def vo(volume, short_lookback=18, long_lookback=36):
    short_ema =  pd.Series.ewm(volume, span=short_lookback).mean()
    long_ema = pd.Series.ewm(volume, span=long_lookback).mean()

    volume_oscillator = (short_ema - long_ema) / long_ema

    return volume_oscillator

In [6]:
# Add technical indicators (for additional features)
df['ema200'] = pd.Series.ewm(df['Mid_Close'], span=200).mean()
df['ema100'] = pd.Series.ewm(df['Mid_Close'], span=100).mean()
df['ema50'] = pd.Series.ewm(df['Mid_Close'], span=50).mean()
df['ema25'] = pd.Series.ewm(df['Mid_Close'], span=25).mean()

df['atr'] = atr(df)
df['rsi'] = rsi(df)
df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['macd'] = pd.Series.ewm(df['Mid_Close'], span=12).mean() - pd.Series.ewm(df['Mid_Close'], span=26).mean()
df['macdsignal'] = pd.Series.ewm(df['macd'], span=9).mean()
df['n_macd'], df['n_macdsignal'] = n_macd(df['macd'], df['macdsignal'])
df['slowk'], df['slowd'] = stoch(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])

df['sar'] = psar(df)

df['vo'] = vo(df['Volume'])

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(df)

                     Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0     2021-12-01 11:05:00   1.27534   1.27534  1.27476    1.27483   1.27553   
1     2021-12-01 11:10:00   1.27485   1.27493  1.27473    1.27487   1.27505   
2     2021-12-01 11:15:00   1.27489   1.27547  1.27484    1.27529   1.27509   
3     2021-12-01 11:20:00   1.27532   1.27532  1.27485    1.27498   1.27553   
4     2021-12-01 11:25:00   1.27496   1.27515  1.27486    1.27486   1.27518   
...                   ...       ...       ...      ...        ...       ...   
10663 2022-01-21 14:35:00   1.25219   1.25350  1.25206    1.25336   1.25237   
10664 2022-01-21 14:40:00   1.25333   1.25413  1.25327    1.25413   1.25350   
10665 2022-01-21 14:45:00   1.25415   1.25421  1.25279    1.25314   1.25434   
10666 2022-01-21 14:50:00   1.25316   1.25420  1.25293    1.25384   1.25337   
10667 2022-01-21 14:55:00   1.25387   1.25392  1.25314    1.25318   1.25404   

       Ask_High  Ask_Low  Ask_Close  Mid_Open  ... 

In [8]:
print(df.loc[df.index[-40:-20], ['Date', 'vo']])

                     Date        vo
10628 2022-01-21 11:40:00 -0.053746
10629 2022-01-21 11:45:00 -0.040673
10630 2022-01-21 11:50:00 -0.020518
10631 2022-01-21 11:55:00  0.054427
10632 2022-01-21 12:00:00  0.072371
10633 2022-01-21 12:05:00  0.083418
10634 2022-01-21 12:10:00  0.095814
10635 2022-01-21 12:15:00  0.070239
10636 2022-01-21 12:20:00  0.055339
10637 2022-01-21 12:25:00  0.037280
10638 2022-01-21 12:30:00  0.035118
10639 2022-01-21 12:35:00  0.045418
10640 2022-01-21 12:40:00  0.028641
10641 2022-01-21 12:45:00  0.014978
10642 2022-01-21 12:50:00  0.001495
10643 2022-01-21 12:55:00 -0.014023
10644 2022-01-21 13:00:00 -0.014185
10645 2022-01-21 13:05:00  0.000887
10646 2022-01-21 13:10:00  0.024043
10647 2022-01-21 13:15:00  0.024237


In [7]:
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]

In [12]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee


def run_simulation(risk_reward_ratio, spread_cutoff, use_tl, min_pips_to_risk, max_pips_to_risk, strict_macd, two_rsi, two_sar):
    max_pips_to_risk /= 10000
    min_pips_to_risk /= 10000
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    n_units = 50000
    trade = None
    lookback = 12

    i = lookback

    while i < len(df):
        curr_date = df.loc[df.index[i], 'Date']
        ema200_2, ema100_2, ema50_2, ema25_2, atr2, sar2, rsi2, mid_open2, mid_close2, mid_low2, mid_high2 = df.loc[df.index[i - 2], ['ema200', 'ema100', 'ema50', 'ema25', 'atr', 'sar', 'rsi', 'Mid_Open', 'Mid_Close', 'Mid_Low', 'Mid_High']]
        ema200_1, ema100_1, ema50_1, ema25_1, atr1, sar1, rsi1, mid_open1, mid_close1, mid_low1, mid_high1 = df.loc[df.index[i - 1], ['ema200', 'ema100', 'ema50', 'ema25', 'atr', 'sar', 'rsi', 'Mid_Open', 'Mid_Close', 'Mid_Low', 'Mid_High']]
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        enough_volatility = spread <= spread_cutoff
        macd2, macdsignal2 = df.loc[df.index[i - 2], ['n_macd', 'n_macdsignal']]
        macd1, macdsignal1 = df.loc[df.index[i - 1], ['n_macd', 'n_macdsignal']]
        macd_vals = [0, macd2, macdsignal2, macd1, macdsignal1]
        stoch_vals = df.loc[df.index[i - 12:i], 'slowk']
        stoch_vals_2 = df.loc[df.index[i - 12:i], 'slowd']
        curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
          df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                              'Ask_Low', 'Ask_Close']]

        emas_buy_signal = ema200_1 < mid_close1
        emas_sell_signal = ema200_1 > mid_close1
        rsi_buy_signal = rsi2 > 50 and rsi1 > 50 if two_rsi else rsi1 > 50
        rsi_sell_signal = rsi2 < 50 and rsi1 < 50 if two_rsi else rsi1 < 50
        stoch_buy_signal = min(stoch_vals) < 20 and min(stoch_vals_2) < 20
        stoch_sell_signal = max(stoch_vals) > 80 and max(stoch_vals_2) > 80
        macd_below_zero = max(macd_vals) == 0 if strict_macd else True
        macd_above_zero = min(macd_vals) == 0 if strict_macd else True
        sar_buy_signal = sar2 < min([mid_open1, mid_close1]) and sar1 < min([mid_open1, mid_close1]) if two_sar else sar1 < min([mid_open1, mid_close1])
        sar_sell_signal = sar2 > max([mid_open1, mid_close1]) and sar1 > max([mid_open1, mid_close1]) if two_sar else sar1 > max([mid_open1, mid_close1])

        if trade is None:
          if macd2 < macdsignal2 and macd1 > macdsignal1 and macd_below_zero and emas_buy_signal and rsi_buy_signal and sar_buy_signal and enough_volatility:
            candle_below_ema200 = True

            # for z in range(i - 4, i):
            #   mid_high, mid_low, curr_ema200 = df.loc[df.index[z], ['Mid_High', 'Mid_Low', 'ema200']]

            #   if min([mid_high, mid_low]) < curr_ema200:
            #     candle_below_ema200 = True
            #     break

            if candle_below_ema200:
              all_buys = True

              if all_buys:
                open_price = float(curr_ask_open)
                pullback = None
                # pullback = min(df.loc[df.index[i - lookback:i], 'Mid_Low'])
                pullback = sar1

                # if open_price - pullback < min_pips_to_risk:
                #   pullback = open_price - min_pips_to_risk

                if open_price - pullback > max_pips_to_risk:
                  pullback = open_price - max_pips_to_risk

                if pullback is not None:
                    stop_loss = round(pullback, 5)
                    # stop_loss = round(pullback - pullback_cushion, 3)

                    if stop_loss < open_price:
                        curr_pips_to_risk = open_price - stop_loss

                        if curr_pips_to_risk <= max_pips_to_risk:
                            stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)
                            # stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 3)

                            trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                              'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                              'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                            n_buys += 1

                            pips_risked.append(curr_pips_to_risk)

          elif macd2 > macdsignal2 and macd1 < macdsignal1 and macd_above_zero and emas_sell_signal and rsi_sell_signal and sar_sell_signal and enough_volatility:
            candle_above_ema200 = True
            
            # for z in range(i - 4, i):
            #   mid_high, mid_low, curr_ema200 = df.loc[df.index[z], ['Mid_High', 'Mid_Low', 'ema200']]

            #   if max([mid_high, mid_low]) > curr_ema200:
            #     candle_above_ema200 = True
            #     break

            if candle_above_ema200:
              all_sells = True

              if all_sells:
                open_price = float(curr_bid_open)
                pullback = None
                # pullback = max(df.loc[df.index[i - lookback:i], 'Mid_High'])
                pullback = sar1

                # if pullback - open_price < min_pips_to_risk:
                #   pullback = open_price + min_pips_to_risk

                if pullback - open_price > max_pips_to_risk:
                  pullback = open_price + max_pips_to_risk

                if pullback is not None:
                    stop_loss = round(pullback, 5)
                    # stop_loss = round(pullback + pullback_cushion, 3)

                    if stop_loss > open_price:
                        curr_pips_to_risk = stop_loss - open_price

                        if curr_pips_to_risk <= max_pips_to_risk:
                            stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)
                            # stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 3)

                            trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                    'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                    'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                            n_sells += 1

                            pips_risked.append(curr_pips_to_risk)

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
          # trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
          trade_amount = -100
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if use_tl:
          if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high - trade['pips_risked'] > trade['stop_loss']:
              trade['stop_loss'] = curr_bid_high - trade['pips_risked']
          # if trade is not None and trade['original_units'] == trade['n_units'] and trade['trade_type'] == 'buy' and curr_bid_high - trade['open_price'] >= trade['pips_risked']:
          #     trade['stop_loss'] = trade['open_price']
          #     reward += trade['pips_risked'] * (trade['n_units'] / 2) * value_per_pip
          #     day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'] / 2)
          #     trade['n_units'] = trade['n_units'] / 2
              

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
          # trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
          trade_amount = 100 * risk_reward_ratio
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
          # trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
          trade_amount = -100
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if use_tl:
          if trade is not None and trade['trade_type'] == 'sell' and trade['pips_risked'] + curr_ask_low < trade['stop_loss']:
            trade['stop_loss'] = trade['pips_risked'] + curr_ask_low
          # if trade is not None and trade['original_units'] == trade['n_units'] and trade['trade_type'] == 'sell' and trade['open_price'] - curr_ask_low >= trade['pips_risked']:
          #     trade['stop_loss'] = trade['open_price']
          #     reward += trade['pips_risked'] * (trade['n_units'] / 2) * value_per_pip
          #     day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'] / 2)
          #     trade['n_units'] = trade['n_units'] / 2

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
          # trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
          trade_amount = 100 * risk_reward_ratio
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        i += 1

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [13]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1.0, 1.5, 2]
# risk_reward_ratio_vals = [2]
spread_cutoffs = [2, 2.5]
# spread_cutoffs = [2.5]
# use_tl_vals = [True, False]
use_tl_vals = [False]
# min_pips_to_risk_vals = [5, 10]
min_pips_to_risk_vals = [10]
max_pips_to_risk_vals = [15, 20, 25, 30]
strict_macd_vals = [True, False]
two_rsi_vals = [True, False]
two_sar_vals = [True, False]
# max_pips_to_risk_vals = [30]
n_possibilities = len(risk_reward_ratio_vals) * len(spread_cutoffs) * len(use_tl_vals) * len(min_pips_to_risk_vals) * len(max_pips_to_risk_vals) * len(strict_macd_vals) * len(two_rsi_vals) * len(two_sar_vals)
all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
  for spread_val in spread_cutoffs:
    for use_tl in use_tl_vals:
      for min_pips in min_pips_to_risk_vals:
        for max_pips in max_pips_to_risk_vals:
          for strict_macd in strict_macd_vals:
            for two_rsi in two_rsi_vals:
              for two_sar in two_sar_vals:
                all_combos.append((risk_reward_ratio, spread_val, use_tl, min_pips, max_pips, strict_macd, two_rsi, two_sar))

# percentage_to_try = 0.5
percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_use_tl_val = None
best_min_pips_val = None
best_max_pips_val = None
best_strict_macd_val = None
best_two_rsi_val = None
best_two_sar_val = None
top_n_results = 20
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, use_tl, min_pips, max_pips, strict_macd, two_rsi, two_sar in combos_to_try:
  reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_simulation(risk_reward_ratio, spread_val, use_tl, min_pips, max_pips, strict_macd, two_rsi, two_sar)
  runs_finished += 1

  print(reward)
  print('Num buys: ' + str(n_sells))
  print('Num sells: ' + str(n_buys))
  print('Num trades: ' + str(n_buys + n_sells))
  print('Num wins: ' + str(n_wins))
  print('Num losses: ' + str(n_losses))
  print('Win streak: ' + str(win_streak))
  print('Loss streak: ' + str(loss_streak))
  # print('Avg pips risked: ' + str(sum(pips_risked) / len(pips_risked)))
  print('Remaining runs: ' + str(n_runs - runs_finished))

  min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None
  if min_item is None or reward > min_item['reward']:
    if min_item is not None:
      best_rewards.remove(min_item)
    best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread': spread_val, 'use_tl': use_tl, 'min_pips': min_pips, 'max_pips': max_pips, 'strict_macd': strict_macd, 'two_rsi': two_rsi, 'two_sar': two_sar})


  if reward > best_reward:
    best_reward = reward
    best_risk_reward = risk_reward_ratio
    best_spread_cutoff = spread_val
    best_use_tl_val = use_tl
    best_min_pips_val = min_pips
    best_max_pips_val = max_pips
    best_strict_macd_val = strict_macd
    best_two_rsi_val = two_rsi
    best_two_sar_val = two_sar

  print('Best reward so far: ' + str(best_reward))
  print()

Num runs: 192

-6228.0
Num buys: 98
Num sells: 119
Num trades: 217
Num wins: 78
Num losses: 139
Win streak: 4
Loss streak: 19
Remaining runs: 191
Best reward so far: -6228.0

-7338.0
Num buys: 167
Num sells: 175
Num trades: 342
Num wins: 135
Num losses: 207
Win streak: 5
Loss streak: 19
Remaining runs: 190
Best reward so far: -6228.0

-7773.0
Num buys: 173
Num sells: 177
Num trades: 350
Num wins: 137
Num losses: 213
Win streak: 5
Loss streak: 19
Remaining runs: 189
Best reward so far: -6228.0

-8553.0
Num buys: 111
Num sells: 133
Num trades: 244
Num wins: 64
Num losses: 180
Win streak: 4
Loss streak: 20
Remaining runs: 188
Best reward so far: -6228.0

-10189.0
Num buys: 210
Num sells: 234
Num trades: 444
Num wins: 115
Num losses: 329
Win streak: 4
Loss streak: 15
Remaining runs: 187
Best reward so far: -6228.0

-15391.0
Num buys: 307
Num sells: 308
Num trades: 615
Num wins: 186
Num losses: 429
Win streak: 5
Loss streak: 18
Remaining runs: 186
Best reward so far: -6228.0

-11741.0
Num b

KeyboardInterrupt: 

In [ ]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best TL val: ' + str(best_use_tl_val))
print('Best min pips to risk: ' + str(best_min_pips_val))
print('Best max pips to risk: ' + str(best_max_pips_val))
print('Best strict macd val: ' + str(best_strict_macd_val))
print('Best two rsi val: ' + str(best_two_rsi_val))
print('Best two sar val: ' + str(best_two_sar_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)